<a href="https://colab.research.google.com/github/kimhyoryeong/ml_edu/blob/master/Copy_of_Chapter_4_4_classification_titanic_example_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "머신러닝 알고리즘 - 분류 Tutorial"
date: 2020-07-16T18:00:47+09:00
tags:
  - "머신러닝"
  - "Python"
  - "Machine Learning"
categories:
  - "머신러닝"
  - "Python"
  - "Machine Learning"
menu: 
  kaggle:
    name: 머신러닝 알고리즘 - 분류 Tutorial
---


## 개요
- Kaggle 대회인 `Titanic'대회를 통해 분류 모형을 만들어본다. 
- 본 강의는 수업 자료의 일부로 작성되었다. 

## I. 사전 준비작업
- `Kaggle API` 설치 및 연동해서 `GCP`에 데이터를 적재하는 것까지 진행한다. 

### (1) Kaggle API 설치
- 구글 코랩에서 `API`를 불러오려면 다음 소스코드를 실행한다. 

In [ ]:
!pip install kaggle

### (2) Kaggle Token 다운로드
- Kaggle에서 API Token을 다운로드 받는다.
- [Kaggle]-[My Account]-[API]-[Create New API Token]을 누르면 `kaggle.json` 파일이 다운로드 된다.
- 이 파일을 바탕화면에 옮긴 뒤, 아래 코드를 실행 시킨다.

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# kaggle.json을 아래 폴더로 옮긴 뒤, file을 사용할 수 있도록 권한을 부여한다. 
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

KeyboardInterrupt: ignored

- 실제 `kaggle.json` 파일이 업로드 되었다는 뜻이다. 

In [ ]:
ls -1ha ~/.kaggle/kaggle.json

### (3) Kaggle 데이터 불러오기
- `Kaggle` 대회 리스트를 불러온다. 

In [ ]:
!kaggle competitions list

- 여기에서 참여하기 원하는 대회의 데이터셋을 불러오면 된다.
- 이번 `basic`강의에서는 `kkbox-churn-prediction-challenge` 데이터를 활용한 데이터 가공과 시각화를 연습할 것이기 때문에 아래와 같이 코드를 실행하여 데이터를 불러온다.

In [ ]:
!kaggle competitions download -c titanic

- 실제 데이터가 잘 다운로드 받게 되었는지 확인한다.

In [ ]:
!ls

## 구글 드라이브 + 코랩연동

In [ ]:
# Google Drive와 마운트
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

In [ ]:
%cd drive/'My Drive'/'Colab Notebooks'/'ml_project'

In [ ]:
%pwd

In [ ]:
%ls

### BigQuery에 데이터 적재
- `sample_submission.csv`, `test.csv`, `train.csv` 데이터를 불러와서 빅쿼리에 적재를 한다. 
- 로컬에서 빅쿼리로 데이터를 Load하는 방법에는 여러가지가 있다.
  + `Local`에서 직접 올리기 (단, 10MB 이하)
  + `Google Stroage` 활용
  + `Pandas` 활용
- `Google Stroage`를 활용하려면 클라우드 수업으로 진행되기 때문에, `Pandas`패키지를 활용한다.
  + `to_gbq`라는 함수를 사용하는데, 이를 위해서는 보통 `pandas-gbq package`패키지를 별도로 설치를 해야한다.
  + 다행히도, 구글 `Colab`에서는 위 패키지는 별도로 설치할 필요가 없다.

In [ ]:
import pandas as pd
from pandas.io import gbq

# import sample_submission file
gender_submission = pd.read_csv('gender_submission.csv')

#woven-passkey-286903:titanic


# Connect to Google Cloud API and Upload DataFrame
gender_submission.to_gbq(destination_table='titanic_classification.gender_submission', 
                  project_id='woven-passkey-286903', 
                  if_exists='replace')


In [ ]:
import pandas as pd
from pandas.io import gbq
# import train file 
train = pd.read_csv('train.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- `column`명을 확인해본다. 

In [ ]:
print(train.columns)

In [ ]:
train.to_gbq(destination_table='titanic_classification.train', 
                  project_id='woven-passkey-286903', 
                  if_exists='replace')

1it [00:03,  3.25s/it]


In [ ]:
# Connect to Google Cloud API and Upload DataFrame
test = pd.read_csv('test.csv')
test.to_gbq(destination_table='titanic_classification.test', 
            project_id='woven-passkey-286903', 
            if_exists='replace')

- 실제 데이터가 들어갔는지 빅쿼리에서 확인한다. 

## II. 데이터 피처공학
- 사이킷런 패키지는 기본적으로 결측치를 허용하지 않기 때문에, 반드시 확인 후, 처리해야 한다. 
- 이번에는 `BigQuery`를 통해 데이터를 불러온다. 
- 주요 데이터 추출을 위한 피처공학에 대해 배워본다. 

### (1) 주요 패키지 불러오기
- 이제 주요 패키지를 불러온다.

In [ ]:
import numpy as np 
import pandas as pd 

from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)

import warnings
warnings.simplefilter(action='ignore')

### (2) 데이터 불러오기

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
# 구글 인증 라이브러리
from google.colab import auth

# 빅쿼리 관련 라이브러리
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

- 먼저 훈련 데이터를 불러온다.

In [ ]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'woven-passkey-286903'
client = bigquery.Client(project=project_id)

df_train = client.query('''
  SELECT 
      * 
  FROM woven-passkey-286903.titanic_classification.train
  ''').to_dataframe()

df_train.shape

- 그 다음은 테스트 데이터를 불러온다. 

In [ ]:
df_test = client.query('''
  SELECT 
      * 
  FROM woven-passkey-286903.titanic_classification.test
  ''').to_dataframe()

df_test.shape

- 아래 코드는 출력 시, 전체 `Column`에 대해 확인할 수 있음

In [ ]:
pd.options.display.max_columns = None 
df_train.head()

- 각 데이터에 대한 설명은 다음을 참조한다. 
  + 참조: https://www.kaggle.com/c/titanic/data

### (3) 결측 데이터 확인

In [ ]:
# data set의 Percent 구하는 함수를 짜보자. 
def check_fill_na(data):
  new_df = data.copy()
  new_df_na = (new_df.isnull().sum() / len(new_df)) * 100
  new_df_na.sort_values(ascending=False).reset_index(drop=True)
  new_df_na = new_df_na.drop(new_df_na[new_df_na == 0].index).sort_values(ascending=False)
  return new_df_na

check_fill_na(df_train)

- 각 데이터에 대한 구체적인 그래프를 작성해본다. 

In [ ]:
ax = df_train["Age"].hist(bins=15, density=True, stacked=True, color='teal', alpha=0.6)
df_train["Age"].plot(kind='density', color='teal')
ax.set(xlabel='Age')
plt.xlim(-10,85)
plt.show()

In [ ]:
print(df_train['Embarked'].value_counts())
sns.countplot(x='Embarked', data=df_train, palette='Set2')
plt.show()

- 결측치에 대한 보간을 진행한다. 
  + `Age`는 중간값 
  + `Embarked`는 `S`로 채웠다. 
  + `Cabin`은 삭제하기로 했다. 

In [ ]:
train_data = df_train.copy()
train_data["Age"].fillna(df_train["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(df_train['Embarked'].value_counts().idxmax(), inplace=True)
train_data.drop('Cabin', axis=1, inplace=True)

In [ ]:
check_fill_na(train_data)

- 위 데이터에 결측치가 없도록 처리하였다. 

### (4) 도출변수
- `SilSp` + `Parch` 변수를 조합하여 혼자 여행을 온 것인지 아닌지 구분하는 도출 변수를 만든 후, 위 변수는 삭제 한다. 

In [ ]:
## Create categorical variable for traveling alone
train_data['TravelAlone']=np.where((train_data["SibSp"]+train_data["Parch"])>0, 0, 1)
train_data.drop('SibSp', axis=1, inplace=True)
train_data.drop('Parch', axis=1, inplace=True)

- `Age`별 생존여부에 관한 그래프를 작성한다. 

In [ ]:
plt.figure(figsize=(20,8))
avg_survival_byage = train_data[["Age", "Survived"]].groupby(['Age'], as_index=False).mean()
g = sns.barplot(x='Age', y='Survived', data=avg_survival_byage, color="LightSeaGreen")
plt.show()

- 그 후에 `under_age`에 대한 도출변수를 추가로 만든다. 

In [ ]:
train_data['IsMinor']=np.where(train_data['Age']<=16, 1, 0)

### (5) 원-핫 인코딩
- 각 `Column` 특히, `문자형 변수`에 대해 원-핫 인코딩을 진행한다. 

In [ ]:
#create categorical variables and drop some variables
training=pd.get_dummies(train_data, columns=["Pclass","Embarked","Sex"])
training.drop('Sex_female', axis=1, inplace=True)
training.drop('PassengerId', axis=1, inplace=True)
training.drop('Name', axis=1, inplace=True)
training.drop('Ticket', axis=1, inplace=True)

final_train = training
final_train.info()

- 위에 했던 작업을 동일하게 `test` 데이터에도 적용한다. 

In [ ]:
test_data = df_test.copy()
test_data["Age"].fillna(df_test["Age"].median(skipna=True), inplace=True)
test_data["Fare"].fillna(df_test["Fare"].median(skipna=True), inplace=True)
test_data.drop('Cabin', axis=1, inplace=True)

test_data['TravelAlone']=np.where((test_data["SibSp"]+test_data["Parch"])>0, 0, 1)

test_data.drop('SibSp', axis=1, inplace=True)
test_data.drop('Parch', axis=1, inplace=True)

test_data['IsMinor']=np.where(test_data['Age']<=16, 1, 0)

testing = pd.get_dummies(test_data, columns=["Pclass","Embarked","Sex"])
testing.drop('Sex_female', axis=1, inplace=True)
testing.drop('PassengerId', axis=1, inplace=True)
testing.drop('Name', axis=1, inplace=True)
testing.drop('Ticket', axis=1, inplace=True)

final_test = testing
final_test.info()

### (6) 피처 선택 (Feature Selection)
- Feature Selection을 통해 학습을 진행한다.
  + 방법론: Recursive Feature Elimination (RFE)
  + 참조: http://scikit-learn.org/stable/modules/feature_selection.html
  + 목적, Backward 방식 중 하나로, 모든 변수를 우선 다 포함시킨 후 반복해서 학습을 진행하면서 중요도가 낮은 변수를 하나씩 제거하는 방식.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

cols = ["Age","Fare","TravelAlone","Pclass_1","Pclass_2","Embarked_C","Embarked_S","Sex_male","IsMinor"] 
X = final_train[cols]
y = final_train['Survived']
model = LogisticRegression()
rfe = RFE(model, 8) # 변수 8개만 선택
rfe = rfe.fit(X, y)
print('Selected features: %s' % list(X.columns[rfe.support_]))

- 이번에는 변수의 갯수에 따라 `classification` 정확도를 시각화 하여 변수의 개수를 정해본다. 

In [ ]:
from sklearn.feature_selection import RFECV
rfecv = RFECV(estimator=LogisticRegression(), step=1, cv=10, scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features: %d" % rfecv.n_features_)
print('Selected features: %s' % list(X.columns[rfecv.support_]))

plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

- 해당 주요 변수를 `Selected_features`로 저장한다. 

In [ ]:
Selected_features = ['Age', 'TravelAlone', 'Pclass_1', 'Pclass_2', 'Embarked_C', 
                     'Embarked_S', 'Sex_male', 'IsMinor']

## III. 머신러닝
- 로지스틱 회귀모형을 통해 머신러닝을 수행한다. 


### (1) 머신러닝 모형 개발
- 데이터셋 분리 부터 모형 개발까지 진행해본다. 

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

# 데이터 셋 분리 
X = final_train[Selected_features]
y = final_train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# 로지스틱 회귀모형
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

idx = np.min(np.where(tpr > 0.95)) # threshold 

plt.figure()
plt.plot(fpr, tpr, color='coral', label='ROC curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot([0,fpr[idx]], [tpr[idx],tpr[idx]], 'k--', color='blue')
plt.plot([fpr[idx],fpr[idx]], [0,tpr[idx]], 'k--', color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

print("Using a threshold of %.3f " % thr[idx] + "guarantees a sensitivity of %.3f " % tpr[idx] +  
      "and a specificity of %.3f" % (1-fpr[idx]) + 
      ", i.e. a false positive rate of %.2f%%." % (np.array(fpr[idx])*100))

- Using a threshold of 0.086 guarantees a sensitivity of 0.959 and a specificity of 0.226, i.e. a false positive rate of 77.36%.

- Using a threshold of 0.672 guarantees a sensitivity of 0.521 and a specificity of 0.953, i.e. a false positive rate of 4.72%.

### (2) 예측 테이블 생성
- 예측 테이블을 만들어 제출한다. 

In [ ]:
final_test['Survived'] = logreg.predict(final_test[Selected_features])
final_test['PassengerId'] = df_test['PassengerId']
submission = final_test[['PassengerId','Survived']]
submission.to_csv("submission.csv", index=False)
print(submission.tail())